### Load and preprocess the data

In [18]:
import mlflow
import numpy as np
import pandas as pd
import os
import json
from sklearn import linear_model
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler
import mlflow.sklearn
from mlflow.models.signature import infer_signature

### Create a new experiment

In [19]:
# Set experiment name
mlflow.set_experiment("Housing Price Prediction")

Traceback (most recent call last):
  File "C:\Users\lapec\anaconda3\Lib\site-packages\mlflow\store\tracking\file_store.py", line 329, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lapec\anaconda3\Lib\site-packages\mlflow\store\tracking\file_store.py", line 427, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lapec\anaconda3\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1373, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lapec\anaconda3\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1366, in _read_helper
    result = read_yaml(root, file_name)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lap

<Experiment: artifact_location='file:///C:/Users/lapec/housing-prices-ml-model/mlruns/766002194513770109', creation_time=1745505050475, experiment_id='766002194513770109', last_update_time=1745505050475, lifecycle_stage='active', name='Housing Price Prediction', tags={}>

### Create a directory to save training data

In [20]:
# Create a directory to save our training data to
def save_training_data(training_data, return_path = True):
    """Save training data to a local subdirectory.

    Training data is typically saved to a central data warehouse. For 
    this project it will be saved to a local subdirectory.

    Args:
        training_data (pd.DataFrame): Pandas DataFrame containing training data
        return_path (bool, optional): Will return the path to the saved training data. Defaults to True.

    Returns:
        str: Path to saved training data
    """
    # Ensure subdirectory exists
    os.makedirs("training_data", exist_ok=True)

    # Write out training data with run_id in file name
    run_id = mlflow.active_run().info.run_id
    training_data.to_csv(f"training_data/{run_id}-training_data.csv", index=False)

    if return_path:
        training_data_path = os.path.abspath(f"training_data/{run_id}-training_data.csv")
        return training_data_path
    else:
        return None

### Split the data into training and validation sets

In [21]:
# Load the JSON lines file
with open('housing_cleaned.json') as f:
    data = [json.loads(line) for line in f]

df = pd.DataFrame(data)

# Define features and target
X = df.drop(["price", "date"], axis=1)
y = df["price"]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Combine features and target into one DataFrame
training_data = pd.concat([X_train, y_train], axis=1)

# Create a Dataset object from the training DataFrame
training_input = mlflow.data.from_pandas(training_data, targets="price")

### Train a Regularized Regression model

In [5]:
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Identify any feature transformations
feature_params = {"preprocessing": "StandardScaler"}

model_params = {
    "alpha": 0.5
}

# Train the Ridge Model
reg = linear_model.Ridge(**model_params)

# Fit the model on the training data
reg.fit(X_train_scaled, y_train)

# Predict on the validation set
y_pred = reg.predict(X_val_scaled)

# Calculate error metrics
mae = mean_absolute_error(y_val, y_pred)
mape = mean_absolute_percentage_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
r2 = r2_score(y_val, y_pred)

# Assemble the metrics into a collection
metrics = {"mae": mae, "mape": mape, "rmse": rmse, "r2": r2}



In [6]:
# Log experiment results and artifacts in MLflow
run_name = "Regularized Regression"
artifact_path = "artifacts"

# Initiate the MLflow run context
with mlflow.start_run(run_name = run_name) as run:
    
    # Log training data and preprocessing details
    mlflow.log_input(training_input, context="training data")
    mlflow.log_params(feature_params)

    # Log the parameters used for the model fit
    mlflow.log_params(model_params)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(sk_model=reg, input_example=X_val_scaled, artifact_path=artifact_path)

### Train a Random Forest model

In [22]:
# Non-scaled data will be used since Random Forest models do not require feature standardization
feature_params = {"preprocessing": "None"}

# Define Random Forest model parameters
model_params = {
    "n_estimators": 200,
    "max_depth": None,
    "min_samples_split": 5,
    "min_samples_leaf": 2,
    "bootstrap": True,
    "oob_score": False,
    "random_state": 888
}

# Train a RandomForest model
rf_model = RandomForestRegressor(**model_params)

# Fit the model on the training data
rf_model.fit(X_train, y_train)

# Evaluate the Random Forest model
y_pred = rf_model.predict(X_val)

# Calculate error metrics
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)
mape = mean_absolute_percentage_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

# Assemble the metrics into a dictionary
metrics = {"rmse": rmse, "mae": mae, "mape": mape, "r2": r2}
metrics


{'rmse': 147552.35164948463,
 'mae': 72868.91266202436,
 'mape': 0.13146711868670297,
 'r2': 0.855985117335387}

In [23]:
# Log experiment results and artifacts in MLflow
run_name = "Random Forest"
artifact_path = "artifacts"

with mlflow.start_run(run_name=run_name) as run:

    # Lot the dataset as an input
    mlflow.log_input(training_input, context="training data")

    # Log the parameters used for the model fit
    for param_name, param_value in model_params.items():
        mlflow.log_param(param_name, param_value)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(sk_model=rf_model, input_example=X_val, artifact_path=artifact_path)

    print("Random Forest model logged in MLflow!")

### Random Forest Hyperparameter search

In [13]:
# Define your param grid
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [None, 6, 20],
    "min_samples_split": [5, 15],
    "min_samples_leaf": [2, 4, 6],
    "bootstrap": [True, False]
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Identify the best parameters and best estimators
best_params = grid_search.best_params_
best_rf = grid_search.best_estimator_

# Predict on the validation set using the best estimator
y_pred_best = best_rf.predict(X_val)

# Calculate error metrics
mae_best = mean_absolute_error(y_val, y_pred_best)
rmse_best = np.sqrt(mean_squared_error(y_val, y_pred_best))
mape_best = mean_absolute_percentage_error(y_val, y_pred_best)
r2_best = r2_score(y_val, y_pred_best)

# Build a dictionary for the error metrics
metrics_best = {"mae": mae_best, "rmse": rmse_best, "mape": mape_best, "r2": r2_best}


Fitting 5 folds for each of 72 candidates, totalling 360 fits


In [17]:
# Log the experiment results and artifacts with MLflow
run_name = "Random Forest Hyperparameter Tuning"
artifact_path = "artifacts"

with mlflow.start_run(run_name=run_name) as run:

    # Lot the dataset and preprocessing details
    mlflow.log_input(training_input, context="training data")
    mlflow.log_params(best_params)

    # Log the best parameters found by GridSearchCV
    mlflow.log_params(best_params)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics_best)

    # Log an instance of the best trained model for later use
    mlflow.sklearn.log_model(sk_model=best_rf, input_example=X_val, artifact_path=artifact_path)
    
    print("Random Forest Hyperparamter Tuning model logged in MLflow!")


C:\Users\lapec\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Random Forest Hyperparamter Tuning model logged in MLflow!


### Experiment using XGBoost (Extreme Gradient Boosting)

In [9]:
import xgboost as xgb
import mlflow.xgboost

# Prepare data for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_val, label=y_val)

# Set hyperparameters for XGBoost
feature_params = {"objective": "reg:squarederror",
                  "max_depth": 3, 
                  "eta": 0.1, 
                  "eval_metric": "rmse" # Default evaluation metric
                 }

# Train the model
evals = [(dtrain, 'train'), (dtest, 'test')]
model = xgb.train(
    feature_params, dtrain,
    num_boost_round=100,
    evals = evals,
    early_stopping_rounds=10
)

# Predict on the validation set using the best estimator
y_pred_best = model.predict(dtest)

# Calculate error metrics
mae_best = mean_absolute_error(y_val, y_pred_best)
rmse_best = np.sqrt(mean_squared_error(y_val, y_pred_best))
mape_best = mean_absolute_percentage_error(y_val, y_pred_best)
r2_best = r2_score(y_val, y_pred_best)

# Build a dictionary for the error metrics
metrics_best = {"mae": mae_best, "rmse": rmse_best, "mape": mape_best, "r2": r2_best}


[0]	train-rmse:340228.50331	test-rmse:366587.84811
[1]	train-rmse:321025.93616	test-rmse:345919.11894
[2]	train-rmse:304190.41558	test-rmse:327832.41792
[3]	train-rmse:289514.74751	test-rmse:312971.56033
[4]	train-rmse:276322.04042	test-rmse:299211.73732
[5]	train-rmse:264256.36079	test-rmse:285930.95812
[6]	train-rmse:253752.63354	test-rmse:274602.34972
[7]	train-rmse:243930.81418	test-rmse:264283.18556
[8]	train-rmse:235125.62211	test-rmse:255326.45848
[9]	train-rmse:227482.77768	test-rmse:248434.39073
[10]	train-rmse:220510.65752	test-rmse:242344.37188
[11]	train-rmse:214249.02315	test-rmse:236150.90289
[12]	train-rmse:208310.79350	test-rmse:229850.49680
[13]	train-rmse:202934.45141	test-rmse:223531.66121
[14]	train-rmse:197940.46421	test-rmse:217710.85242
[15]	train-rmse:192987.29563	test-rmse:212188.16277
[16]	train-rmse:188958.61531	test-rmse:208471.56680
[17]	train-rmse:185028.07251	test-rmse:204470.44486
[18]	train-rmse:181799.24682	test-rmse:202645.80537
[19]	train-rmse:178735

In [10]:
# Log the experiment results and artifacts with MLflow
from mlflow.models.signature import infer_signature

run_name = "XGBoost"

# Start an MLflow run
with mlflow.start_run(run_name=run_name) as run:

    # Log model parameters
    mlflow.log_params(feature_params)

    # Log metrics
    mlflow.log_metrics(metrics_best)

    # Convert all integer columns to float before logging
    X_val_fixed = X_val.astype({col: 'float64' for col in X_val.select_dtypes('int').columns})
    
    # Infer signature
    signature = infer_signature(X_val_fixed, y_pred_best)

    # Prepare input example (first row of X_val)
    input_example = X_val_fixed.iloc[:1] if hasattr(X_val, "iloc") else X_val[:1]

    # Log the model
    mlflow.xgboost.log_model(
        xgb_model=model,
        artifact_path="model",
        signature=signature,
        input_example=input_example
    )

    print("XGBoost model logged in MLflow!")

C:\Users\lapec\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [10:34:36] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


XGBoost model logged in MLflow!


### K-Nearest Neighbors (KNN)
KNN can be used for regression, where the output value for a test point is the average of the nearest neighbors. It can be a good option if the dataset has local patterns that are significant.

In [20]:
# from sklearn.neighbors import KNeighborsRegressor

# # Initialize the model
# model = KNeighborsRegressor(n_neighbors=5)

# # Train the model
# model.fit(X_train, y_train)

# # Set experiment
# mlflow.set_experiment("housing-price-kneighbors")

# # Log the model to MLflow
# with mlflow.start_run():
#     mlflow.log_params({'n_neighbors': 5})
#     mlflow.sklearn.log_model(model, "model")


KNeighborsRegressor()

### Training for Neural Network
Need to load NN specific dataset in and train the neural network on it - using learned embeddings with an embedding layer for Zip Code

In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Input, Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
import mlflow
import mlflow.tensorflow

# Assume this is your preprocessed DataFrame (from earlier)
df = pd.read_csv("housing_for_nn.csv")

# Define inputs
target_col = 'price'
ordinal_cols = ['condition', 'grade', 'view', 'floors']
numeric_cols = [
    'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
    'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
    'lat', 'long', 'sqft_living15', 'sqft_lot15'
]
zipcode_col = 'zipcode'

# Separate features and target
X = df[numeric_cols + ordinal_cols + [zipcode_col]]
y = df[target_col].values

# Split into train/val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Separate numeric + ordinal and zipcode
X_train_numeric = X_train[numeric_cols + ordinal_cols].values
X_train_zip = X_train[zipcode_col].values
X_val_numeric = X_val[numeric_cols + ordinal_cols].values
X_val_zip = X_val[zipcode_col].values

# Normalize non-zipcode features
scaler = StandardScaler()
X_train_numeric = scaler.fit_transform(X_train_numeric)
X_val_numeric = scaler.transform(X_val_numeric)

# Build a lookup for zipcodes → integer index
unique_zipcodes = df[zipcode_col].unique()
zipcode_lookup = {zip_code: idx for idx, zip_code in enumerate(unique_zipcodes)}

# Map zipcodes to index values
X_train_zip = X_train[zipcode_col].map(zipcode_lookup).values
X_val_zip = X_val[zipcode_col].map(zipcode_lookup).values

# Neural Net parameters
embedding_dim = 4
n_zipcodes = len(zipcode_lookup)
hidden_units = [64, 32]

# Build model
input_numeric = Input(shape=(X_train_numeric.shape[1],), name='numeric')
input_zipcode = Input(shape=(1,), dtype='int32', name='zipcode')
zipcode_embedding = layers.Embedding(input_dim=n_zipcodes + 1, output_dim=embedding_dim)(input_zipcode)
zipcode_embedding = layers.Flatten()(zipcode_embedding)

x = layers.Concatenate()([input_numeric, zipcode_embedding])
for units in hidden_units:
    x = layers.Dense(units, activation='relu')(x)
output = layers.Dense(1)(x)

model = Model(inputs=[input_numeric, input_zipcode], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(
    {'numeric': X_train_numeric, 'zipcode': X_train_zip},
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=({'numeric': X_val_numeric, 'zipcode': X_val_zip}, y_val),
    verbose=1
)

# Predict and compute metrics
y_pred = model.predict({'numeric': X_val_numeric, 'zipcode': X_val_zip}).flatten()

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)
mape = mean_absolute_percentage_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

metrics = {"rmse": rmse, "mae": mae, "mape": mape, "r2": r2}
params = {
    "embedding_dim": embedding_dim,
    "hidden_layers": hidden_units,
    "optimizer": "adam",
    "epochs": 50,
    "batch_size": 32
}

Epoch 1/50
541/541 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 414086201344.0000 - mae: 536013.0000 - val_loss: 443100430336.0000 - val_mae: 542473.1875
Epoch 2/50
541/541 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 386463334400.0000 - mae: 516051.0938 - val_loss: 337783750656.0000 - val_mae: 455802.4688
Epoch 3/50
541/541 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 269353549824.0000 - mae: 394995.6250 - val_loss: 149170798592.0000 - val_mae: 228887.1094
Epoch 4/50
541/541 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 101336563712.0000 - mae: 180862.1719 - val_loss: 76891553792.0000 - val_mae: 138490.5938
Epoch 5/50
541/541 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 54446333952.0000 - mae: 127436.9609 - val_loss: 59758940160.0000 - val_mae: 126957.9844
Epoch 6/50
541/541 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 49116778496.0000 - mae: 119722.5859 - val_loss: 52242923520.0000 - val_mae: 118889.1797
Epoch 7/50
541/541 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 41053593600.0000 - mae: 112442.5781 - val_los

In [12]:
# MLflow logging
run_name = "Neural Network with Zipcode Embedding"
artifact_path = "nn_model"

# Since there are two inputs for this model, the input_example will need to be defined to include both
numeric_example = X_val_numeric[:1]  # shape (1, num_features)
zipcode_example = X_val_zip[:1]      # shape (1,)

input_example = {
    "numeric": numeric_example,
    "zipcode": zipcode_example
}

with mlflow.start_run(run_name=run_name) as run:
    # Log input metadata (optional)
    # mlflow.log_input(X_train, context="training data")  # uncomment if using MLflow 2.0+ with data tracking
    
    for name, val in params.items():
        mlflow.log_param(name, val)
    
    mlflow.log_metrics(metrics)
    
    # Save model using MLflow
    mlflow.tensorflow.log_model(model=model, 
                                input_example=input_example, 
                                artifact_path=artifact_path,
                                signature=mlflow.models.infer_signature(
                                    {"numeric": X_val_numeric, "zipcode": X_val_zip})
                                )

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
